In [1]:
from qiskit.opflow import X, Y, Z, I, MatrixEvolution, PauliTrotterEvolution, PauliExpectation, StateFn
from qiskit.algorithms import VQE, QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit import Aer, QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, EfficientSU2, ZFeatureMap
from qiskit.circuit import Parameter
from qiskit.opflow import CircuitSampler
import numpy as np

import warnings

warnings.filterwarnings('ignore')

# Hamiltonian time evolution simulation

$$ \hat{H} = \frac{1}{2} \left( \hat{I}\otimes \hat{I} +  \hat{\sigma}_x \otimes \hat{\sigma}_x + \hat{\sigma}_y \otimes \hat{\sigma}_y + \hat{\sigma}_z \otimes \hat{\sigma}_z \right) $$

In [2]:
operator_H = ((I ^ I) + (X ^ X) + (Y ^ Y) + (Z ^ Z))*1/2

In [3]:
print(operator_H)

0.5 * II
+ 0.5 * XX
+ 0.5 * YY
+ 0.5 * ZZ


In [4]:
H_matrix = operator_H.to_matrix()

print(H_matrix)

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


In [5]:
t = Parameter('t')

time_H = (t * operator_H).exp_i()

In [6]:
print(time_H)

e^(-i*1.0*t * (
  0.5 * II
  + 0.5 * XX
  + 0.5 * YY
  + 0.5 * ZZ
))


In [7]:
evolution_time = 0.5

mat_evo = MatrixEvolution()

result_mat_exp = mat_evo.convert(time_H)
result_mat_exp = result_mat_exp.bind_parameters({t: evolution_time})

print(result_mat_exp.to_matrix())
print(result_mat_exp)

Evolved Hamiltonian is not composed of only MatrixOps, converting to Matrix representation, which can be expensive.


[[0.87758256-0.47942554j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.87758256+0.j         0.        -0.47942554j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.47942554j 0.87758256+0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.87758256-0.47942554j]]
     ┌──────────────┐
q_0: ┤0             ├
     │  Hamiltonian │
q_1: ┤1             ├
     └──────────────┘


In [8]:
observable = X ^ X

observable_measurement = StateFn(observable).adjoint()

eigenvalues, eigenstates = np.linalg.eigh(operator_H.to_matrix())
initial_state = StateFn(eigenstates[0])

print(initial_state.to_circuit_op())

CircuitStateFn(
     ┌──────────────────────┐
q_0: ┤0                     ├
     │  Initialize(0,0,1,0) │
q_1: ┤1                     ├
     └──────────────────────┘
)


In [9]:
evo_and_measure = observable_measurement @ time_H @ initial_state

print(evo_and_measure)

ComposedOp([
  OperatorMeasurement(XX),
  e^(-i*1.0*t * (
    0.5 * II
    + 0.5 * XX
    + 0.5 * YY
    + 0.5 * ZZ
  )),
  VectorStateFn(Statevector([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
              dims=(2, 2)))
])


In [10]:
pauli_trot = PauliTrotterEvolution(trotter_mode='trotter', reps = 1)

trotterized_op = pauli_trot.convert(evo_and_measure)

print(trotterized_op)

ComposedOp([
  OperatorMeasurement(XX),
       ┌──────────────────────────────────────┐
  q_0: ┤0                                     ├
       │  exp(-it (II + XX + YY + ZZ))(1.0*t) │
  q_1: ┤1                                     ├
       └──────────────────────────────────────┘,
  VectorStateFn(Statevector([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
              dims=(2, 2)))
])


In [11]:
# Run Hamiltoninan in ideal device
evo_time_points = [5.5, 0.75]

hamiltonian_trotter_expectations = trotterized_op.bind_parameters({t: evo_time_points})

print(f"Observable at time {evo_time_points}: {np.round(hamiltonian_trotter_expectations.eval(), 3)}")

Observable at time [5.5, 0.75]: [ 0.+0.j -0.+0.j]


In [12]:
# Run Hamiltoninan in simulatoror device

sampler = CircuitSampler(backend=Aer.get_backend("qasm_simulator"))

sampled_trotter_exp_op = sampler.convert(hamiltonian_trotter_expectations)
sampled_trotter_energies = sampled_trotter_exp_op.eval()

print("Energies: ", np.round(np.real(sampled_trotter_energies), 3))

Energies:  [1.    0.997]


# VQE for Hamiltonian

In [13]:
backend = Aer.get_backend('statevector_simulator')

In [14]:
quantum_instance = QuantumInstance(backend, 
                                   shots = 1024, 
                                   seed_simulator = algorithm_globals.random_seed, 
                                   seed_transpiler = algorithm_globals.random_seed)

In [15]:
#ansatz = RealAmplitudes(num_qubits = 1, reps = 4)
ansatz = ZFeatureMap(1, reps = 4)
#ansatz = EfficientSU2(num_qubits = 1, reps = 4)

In [16]:
optimizer = COBYLA(maxiter = 120, tol = 0.001)

vqe = VQE(ansatz=ansatz, quantum_instance=quantum_instance, optimizer=optimizer)

In [17]:
minimum_energy = vqe.compute_minimum_eigenvalue(operator_H).eigenvalue

print("Minimum energy eigenvalue:", minimum_energy)

Minimum energy eigenvalue: (0.18195580130920275+0j)


# QAOA for Hamiltonian

In [18]:
qaoa = QAOA(optimizer, quantum_instance=Aer.get_backend('statevector_simulator'))

result = qaoa.compute_minimum_eigenvalue(operator_H)

minimum_energy = result.eigenstate

print("Minimum energy eigenvalue:\n\n", minimum_energy)

Minimum energy eigenvalue:

 [-0.49602987-0.0628838j -0.49602987-0.0628838j -0.49602987-0.0628838j
 -0.49602987-0.0628838j]
